In [1]:
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
from random import randint
import re

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
data = open('001ssb.txt', 'r')
text = data.read()
text[0:2000]

'A Game Of Thrones \nBook One of A Song of Ice and Fire \nBy George R. R. Martin \nPROLOGUE \n"We should start back," Gared urged as the woods began to grow dark around them. "The wildlings are \ndead." \n"Do the dead frighten you?" Ser Waymar Royce asked with just the hint of a smile. \nGared did not rise to the bait. He was an old man, past fifty, and he had seen the lordlings come and go. \n"Dead is dead," he said. "We have no business with the dead." \n"Are they dead?" Royce asked softly. "What proof have we?" \n"Will saw them," Gared said. "If he says they are dead, that\'s proof enough for me." \nWill had known they would drag him into the quarrel sooner or later. He wished it had been later rather \nthan sooner. "My mother told me that dead men sing no songs," he put in. \n"My wet nurse said the same thing, Will," Royce replied. "Never believe anything you hear at a woman\'s \ntit. There are things to be learned even from the dead." His voice echoed, too loud in the twilit fores

In [4]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)
    #Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s", ' ', sentence)
    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence.lower()

In [5]:
text_processed = preprocess_text(text)
text_processed = text_processed[0:10000]

In [6]:
from nltk.tokenize import word_tokenize

text_words = (word_tokenize(text_processed))
n_words = len(text_words)
unique_words = len(set(text_words))

print('Total words: %d' % n_words)
print('Unique words: %d' % unique_words)

Total words: 1915
Unique words: 705


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=11318)
tokenizer.fit_on_texts(text_words)

In [8]:
vocab_size = len(tokenizer.word_index) + 1
word_2_index = tokenizer.word_index

In [9]:
print(text_words[500])
print(word_2_index[text_words[500]])

especially
347


In [10]:
input_sequence = []
output_words = []
input_seq_length = 100

for i in range(0, n_words - input_seq_length, 1):
    in_seq = text_words[i: i + input_seq_length]
    out_seq = text_words[i + input_seq_length]
    input_sequence.append([word_2_index[word] for word in in_seq])
    output_words.append(word_2_index[out_seq])

In [11]:
print(input_sequence[76])

[7, 28, 18, 37, 19, 258, 27, 1, 24, 44, 32, 24, 15, 51, 259, 66, 150, 37, 18, 8, 52, 40, 12, 28, 86, 7, 260, 32, 44, 24, 13, 150, 67, 20, 53, 8, 4, 261, 32, 112, 262, 33, 113, 1, 263, 151, 87, 152, 7, 264, 3, 4, 21, 152, 265, 54, 151, 55, 266, 153, 53, 13, 24, 46, 267, 19, 268, 7, 154, 14, 55, 269, 270, 28, 1, 155, 271, 8, 15, 272, 68, 273, 274, 10, 156, 22, 157, 275, 29, 44, 158, 6, 114, 276, 277, 47, 1, 24, 9, 278]


In [12]:
X = np.reshape(input_sequence, (len(input_sequence), input_seq_length, 1))
X = X / float(vocab_size)

y = to_categorical(output_words)

In [13]:
print("X shape:\t", X.shape)
print("y shape:\t", y.shape)

X shape:	 (1815, 100, 1)
y shape:	 (1815, 706)


In [14]:
X

array([[[0.34135977],
        [0.3427762 ],
        [0.00708215],
        ...,
        [0.05665722],
        [0.01699717],
        [0.03966006]],

       [[0.3427762 ],
        [0.00708215],
        [0.34419263],
        ...,
        [0.01699717],
        [0.03966006],
        [0.12181303]],

       [[0.00708215],
        [0.34419263],
        [0.34560907],
        ...,
        [0.03966006],
        [0.12181303],
        [0.00991501]],

       ...,

       [[0.25212465],
        [0.01133144],
        [0.05382436],
        ...,
        [0.00566572],
        [0.18696884],
        [0.02974504]],

       [[0.01133144],
        [0.05382436],
        [0.19971671],
        ...,
        [0.18696884],
        [0.02974504],
        [0.99716714]],

       [[0.05382436],
        [0.19971671],
        [0.00424929],
        ...,
        [0.02974504],
        [0.99716714],
        [0.01983003]]])

## Model Training

In [15]:
model = Sequential()
model.add(LSTM(800, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(LSTM(600, return_sequences=True))
model.add(LSTM(800))
model.add(Dense(y.shape[1], activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 800)          2566400   
                                                                 
 lstm_1 (LSTM)               (None, 100, 600)          3362400   
                                                                 
 lstm_2 (LSTM)               (None, 800)               4483200   
                                                                 
 dense (Dense)               (None, 706)               565506    
                                                                 
Total params: 10977506 (41.88 MB)
Trainable params: 10977506 (41.88 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [17]:
model

In [18]:
model.fit(X, y, batch_size=64, epochs=100, verbose=1)

Epoch 1/100
29/29 [==============================] - 14s 142ms/step - loss: 6.2525
Epoch 2/100
29/29 [==============================] - 4s 141ms/step - loss: 5.8902
Epoch 3/100
29/29 [==============================] - 4s 141ms/step - loss: 5.8369
Epoch 4/100
29/29 [==============================] - 4s 142ms/step - loss: 5.8162
Epoch 5/100
29/29 [==============================] - 4s 144ms/step - loss: 5.8139
Epoch 6/100
29/29 [==============================] - 4s 144ms/step - loss: 5.8095
Epoch 7/100
29/29 [==============================] - 4s 144ms/step - loss: 5.8033
Epoch 8/100
29/29 [==============================] - 4s 146ms/step - loss: 5.8058
Epoch 9/100
29/29 [==============================] - 4s 147ms/step - loss: 5.8019
Epoch 10/100
29/29 [==============================] - 4s 151ms/step - loss: 5.7980
Epoch 11/100
29/29 [==============================] - 4s 155ms/step - loss: 5.7973
Epoch 12/100
29/29 [==============================] - 4s 150ms/step - loss: 5.7956
Epoch 13/100

In [19]:
random_seq_index = np.random.randint(0, len(input_sequence) - 1)
random_seq = input_sequence[random_seq_index]

index_2_word = dict(map(reversed, word_2_index.items()))

word_sequence = [index_2_word[value] for value in random_seq]

print(' '.join(word_sequence))

knight turned back to his grizzled man at arms frostfallen leaves whispered past them and royce destrier moved restlessly what do you think might have killed these men gared ser waymar asked casually he adjusted the drape of his long sable cloak it was the cold gared said with iron certainty saw men freeze last winter and the one before when was half boy everyone talks about snows forty foot deep and how the ice wind comes howling out of the north but the real enemy is the cold it steals up on you quieter than will and at first


In [20]:
for i in range(100):
  int_sample = np.reshape(random_seq, (1, len(random_seq), 1))
  int_sample = int_sample / float(vocab_size)

  predicted_word_index = model.predict(int_sample, verbose = 1)

  predicted_word_id = np.argmax(predicted_word_index)
  seq_in = [index_2_word[index] for index in random_seq]

  word_sequence.append(index_2_word[predicted_word_id])

  random_seq.append(predicted_word_id)
  random_seq = random_seq[1: len(random_seq)]

  final_output = ""
  for word in word_sequence:
    final_output = final_output + " " + word

print(final_output)

1/1 [==============================] - 0s 22ms/step
 knight turned back to his grizzled man at arms frostfallen leaves whispered past them and royce destrier moved restlessly what do you think might have killed these men gared ser waymar asked casually he adjusted the drape of his long sable cloak it was the cold gared said with iron certainty saw men freeze last winter and the one before when was half boy everyone talks about snows forty foot deep and how the ice wind comes howling out of the north but the real enemy is the cold it steals up on you quieter than will and at first the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
